In [8]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../')
from modules import utils, constants
from modules.env import AnemiaEnv
import stable_baselines
from stable_baselines.common.vec_env import DummyVecEnv

In [9]:
seed = constants.SEED

In [10]:
def load_model(filepath, model_type, env):
    if model_type.lower() == 'dqn3': #dqn3
        return stable_baselines3.DQN.load(filepath, env=env)
    elif model_type.lower() == 'dqn': #dqn
        return stable_baselines.DQN.load(filepath, env=env)
    else:
        raise ValueError(f'Unknown model type {model_type}')

In [11]:
def create_test_df(model_path, model_type, X_test, y_test):
    testing_env = DummyVecEnv([lambda: AnemiaEnv(X_test, y_test, random=False)])
#     testing_env = LupusEnv(X_test, y_test, random=False)
#     testing_env.seed(SEED)
    model = load_model(model_path, model_type, testing_env)
    test_df = pd.DataFrame()
    try:
        while True:
            obs, done = testing_env.reset(), False
            while not done:
                action, states = model.predict(obs, deterministic=True)
                obs, rew, done, info = testing_env.step(action)
                if done==True:
                    test_df = test_df.append(info, ignore_index=True)

    except StopIteration:
        pass
    return test_df

#### The data

In [12]:
testing_df = pd.read_csv('../data/test_set_constant.csv')
testing_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,7.116363,-1.000000,3.781573,2.738413,-1.000000,95.904198,68.457895,2.226085,0,1.892912,39.808550,110.329197,64.404350,21.654404,73.787009,21.349089,-1.000000,5
1,8.125320,92.230003,4.231419,1.188039,143.365567,104.057204,204.747831,2.342554,0,0.652614,13.478089,-1.000000,32.705481,-1.000000,43.520272,24.375961,142.815207,1
2,11.309450,38.324563,-1.000000,-1.000000,455.077909,76.402602,-1.000000,4.440732,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,33.928350,-1.000000,4
3,13.763858,253.513394,2.262606,0.551444,453.772884,82.781943,90.101466,4.987993,0,0.853521,104.005514,34.639227,0.963866,22.083012,88.891838,41.291574,19.856071,0
4,11.464002,-1.000000,-1.000000,-1.000000,320.964653,104.287127,-1.000000,3.297819,0,1.163516,121.616315,105.895897,-1.000000,9.337462,-1.000000,34.392007,-1.000000,7


In [13]:
X_test = testing_df.iloc[:, 0:-1]
y_test = testing_df.iloc[:, -1]
X_test, y_test = np.array(X_test), np.array(y_test)
X_test.shape, y_test.shape

((14000, 17), (14000,))

In [14]:
# test_df = create_test_df('../models/logs/sb/dueling_ddqn/best_acc_model.zip', 'dqn', X_test, y_test)
test_df = create_test_df('../models/dueling_double_dqn_basic_anemia_diagnosis.zip', 'dqn', X_test, y_test)
test_df.head()








Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where









,index,episode_length,reward,y_pred,y_actual,trajectory,terminated,is_success,terminal_observation
0,0,4,-1,7,5,"[serum_iron, mcv, glucose, Inconclusive diagno...",True,False,"[-1.0, -1.0, -1.0, -1.0, -1.0, 95.9042, 68.457..."
1,2,2,-1,7,4,"[serum_iron, Inconclusive diagnosis]",True,False,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1...."
2,4,2,-1,7,7,"[serum_iron, Inconclusive diagnosis]",True,True,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1...."
3,6,2,-1,7,1,"[serum_iron, Inconclusive diagnosis]",True,False,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1...."
4,8,2,-1,7,1,"[serum_iron, Inconclusive diagnosis]",True,False,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1...."


In [15]:
success_rate, f1, roc_auc = utils.get_metrics(test_df)
success_rate, f1, roc_auc

(9.77142857142857, 2.225403435710567, 50.0)

In [16]:
test_df.episode_length.min(), test_df.episode_length.mean(), test_df.episode_length.max()

(2, 3.34, 10)

In [ ]:
# test_df.to_csv('../test_dfs/23_03_23/dueling_double_dqn_basic_test_df.csv', index=False)